In [1]:
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, r2_score


In [2]:
df = pd.read_csv('Agricultural_Yield_Prediction.csv')

In [3]:
df.head(10)

,Crop,Crop_Year,Season,State,Area,Production,Annual_Rainfall,Fertilizer,Pesticide,Yield
0,Arecanut,1997,Whole Year,Assam,73814.0,56708,2051.4,7024878.38,22882.34,0.796087
1,Arhar/Tur,1997,Kharif,Assam,6637.0,4685,2051.4,631643.29,2057.47,0.710435
2,Castor seed,1997,Kharif,Assam,796.0,22,2051.4,75755.32,246.76,0.238333
3,Coconut,1997,Whole Year,Assam,19656.0,126905000,2051.4,1870661.52,6093.36,5238.051739
4,Cotton(lint),1997,Kharif,Assam,1739.0,794,2051.4,165500.63,539.09,0.420909
5,Dry chillies,1997,Whole Year,Assam,13587.0,9073,2051.4,1293074.79,4211.97,0.643636
6,Gram,1997,Rabi,Assam,2979.0,1507,2051.4,283511.43,923.49,0.465455
7,Jute,1997,Kharif,Assam,94520.0,904095,2051.4,8995468.40,29301.20,9.919565
8,Linseed,1997,Rabi,Assam,10098.0,5158,2051.4,961026.66,3130.38,0.461364
9,Maize,1997,Kharif,Assam,19216.0,14721,2051.4,1828786.72,5956.96,0.615652


In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 19689 entries, 0 to 19688
Data columns (total 10 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   Crop             19689 non-null  object 
 1   Crop_Year        19689 non-null  int64  
 2   Season           19689 non-null  object 
 3   State            19689 non-null  object 
 4   Area             19689 non-null  float64
 5   Production       19689 non-null  int64  
 6   Annual_Rainfall  19689 non-null  float64
 7   Fertilizer       19689 non-null  float64
 8   Pesticide        19689 non-null  float64
 9   Yield            19689 non-null  float64
dtypes: float64(5), int64(2), object(3)
memory usage: 1.5+ MB


In [5]:
df.describe()

,Crop_Year,Area,Production,Annual_Rainfall,Fertilizer,Pesticide,Yield
count,19689.000000,1.968900e+04,1.968900e+04,19689.000000,1.968900e+04,1.968900e+04,19689.000000
mean,2009.127584,1.799266e+05,1.643594e+07,1437.755177,2.410331e+07,4.884835e+04,79.954009
std,6.498099,7.328287e+05,2.630568e+08,816.909589,9.494600e+07,2.132874e+05,878.306193
min,1997.000000,5.000000e-01,0.000000e+00,301.300000,5.417000e+01,9.000000e-02,0.000000
25%,2004.000000,1.390000e+03,1.393000e+03,940.700000,1.880146e+05,3.567000e+02,0.600000
50%,2010.000000,9.317000e+03,1.380400e+04,1247.600000,1.234957e+06,2.421900e+03,1.030000
75%,2015.000000,7.511200e+04,1.227180e+05,1643.700000,1.000385e+07,2.004170e+04,2.388889
max,2020.000000,5.080810e+07,6.326000e+09,6552.700000,4.835407e+09,1.575051e+07,21105.000000


In [6]:
correlation = df.corr(numeric_only = True)
correlation


,Crop_Year,Area,Production,Annual_Rainfall,Fertilizer,Pesticide,Yield
Crop_Year,1.000000,-0.035686,0.003366,-0.011187,0.011169,-0.004657,0.002539
Area,-0.035686,1.000000,0.037441,-0.106054,0.973255,0.973479,0.001858
Production,0.003366,0.037441,1.000000,0.029879,0.039799,0.035171,0.570809
Annual_Rainfall,-0.011187,-0.106054,0.029879,1.000000,-0.109734,-0.097657,0.020761
Fertilizer,0.011169,0.973255,0.039799,-0.109734,1.000000,0.954991,0.002862
Pesticide,-0.004657,0.973479,0.035171,-0.097657,0.954991,1.000000,0.001782
Yield,0.002539,0.001858,0.570809,0.020761,0.002862,0.001782,1.000000


In [7]:
fig = px.imshow(correlation, text_auto = '.3f', aspect = 'auto',
               title = '<b>Correlation Matrix</b>', color_continuous_scale = 'tropic')
fig.update_layout(title_font_color = 'olive', font_color = 'fuchsia')
fig.show()


In [8]:
fig = make_subplots(rows = 5, cols = 1,
                    subplot_titles = ['<b>Production vs Yield</b>', '<b>Area vs Yield</b>',
                                     '<b>Annual Rainfall vs Yield</b>', '<b>Fertilizer vs Yield</b>',
                                     '<b>Pesticide vs Yield</b>'])

fig.add_trace(go.Scattergl(x = df['Production'], y = df['Yield'], mode = 'markers',
                          marker_line_color = 'darkblue', marker_line_width = 1,
                          name = 'Production'), row = 1, col = 1)

fig.add_trace(go.Scattergl(x = df['Area'], y = df['Yield'], mode = 'markers',
                           marker_color = 'coral', marker_line_color = 'darkslategray',
                           marker_line_width = 1, name = 'Area'), row = 2, col = 1)

fig.add_trace(go.Scattergl(x = df['Annual_Rainfall'], y = df['Yield'], mode = 'markers',
                           marker_color = 'beige', marker_line_width = 1,
                           marker_line_color = 'coral',
                           name = 'Annual Rainfall'), row = 3, col = 1)

fig.add_trace(go.Scattergl(x = df['Fertilizer'], y = df['Yield'], mode = 'markers',
                            marker_line_width = 1, name = 'Fertilizer'),row = 4, col = 1)

fig.add_trace(go.Scattergl(x = df['Pesticide'], y = df['Yield'], mode = 'markers',
                            marker_line_width = 1, name = 'Pesticide'), row = 5, col = 1)
fig.update_xaxes(type = 'log')
fig.update_xaxes(title = 'Production', row = 1, col = 1)
fig.update_xaxes(title = 'Area', row = 2, col = 1)
fig.update_xaxes(title = 'Annual Rainfall', row = 3, col = 1)
fig.update_xaxes(title = 'Fertilizer', row = 4, col = 1)
fig.update_xaxes(title = 'Pesticide', row = 5, col = 1)

fig.update_yaxes(type = 'log', title = 'Yield')
fig.update_layout(title = '<b>Impact of Various Factors on Crop Yield</b>',
                  title_font_color = 'sienna', legend_font_color = 'saddlebrown',
                  height = 1700)
fig.show()

In [9]:
X = df.drop(['Crop', 'Crop_Year', 'Season', 'State', 'Yield','Production'], axis = 1)
y = df['Yield']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2 ,random_state = 42)
print(X_train.shape, y_train.shape)
print(X_test.shape, y_test.shape)


(15751, 4) (15751,)
(3938, 4) (3938,)


In [10]:
lr = LinearRegression()
lr.fit(X_train, y_train)


LinearRegression()

In [18]:
pred_lr = lr.predict(X_test)
r2_lr = r2_score(y_test, pred_lr)
print("r2 Score of Linear Regression: ", r2_lr)


r2 Score of Linear Regression:  0.0005353688048722605


In [26]:
comparison = pd.DataFrame()


In [27]:
comparison['Type'] = ['Linear Regression']
comparison['r2 Score'] = [r2_lr]
comparison



,Type,r2 Score
0,Linear Regression,0.000535


In [28]:
print(pred_lr)


[ 67.0526911   80.29278596 133.85452223 ...  83.31126157  79.98834397
  71.50052441]
